<a href="https://colab.research.google.com/github/harenlin/Simple-Text-Generation/blob/main/Embedding_Layer_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Flatten, Input, Embedding

# define documents
docs = ['Well done!', 'Good work', 'Great effort', 'nice work', 'Excellent!',
		'Weak', 'Poor effort!', 'not good', 'poor work', 'Could have done better.']
# define class labels
labels = [1,1,1,1,1,0,0,0,0,0]

# prepare tokenizer
from keras.preprocessing.text import Tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=50,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n') # encode sequences of words with index
tokenizer.fit_on_texts(docs) # create a hash map of numbers and words, word2idx & idx2word
sequences = tokenizer.texts_to_sequences(docs) # shape = (# of docs, length of text)
print(sequences)
vocabulary_size = len(tokenizer.word_counts)
print("The size of vocab.txt is", vocabulary_size)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]
The size of vocab.txt is 14


In [2]:
# p.s. Member Variables of Tokenizer()
# (1) document_count = number of documents
print(tokenizer.document_count) 
print(len(sequences))
# (2) word_index = word2idx (index starting from 1)
# (3) index_word = idx2word
print(tokenizer.index_word[5])
print(tokenizer.word_index[tokenizer.index_word[5]])
# (4) word_counts = dictionary of the each word and its occurence
print(tokenizer.word_counts)

10
10
poor
5
OrderedDict([('well', 1), ('done', 2), ('good', 2), ('work', 3), ('great', 1), ('effort', 2), ('nice', 1), ('excellent', 1), ('weak', 1), ('poor', 2), ('not', 1), ('could', 1), ('have', 1), ('better', 1)])


In [3]:
# since the max sequence length of the corpus is 4 (doc[9]), 
# we are going to made the max_seq_len = 4 -> PADDING
from keras.preprocessing.sequence import pad_sequences
max_seq_len = 4
sequences = pad_sequences(sequences, maxlen=max_seq_len, padding='post')
print(sequences)

# define simple model to understand what Embedding do
def embedding_model(vocabulary_size, hidden_dim, max_seq_len):
    input = Input(shape=(max_seq_len))
    x = Embedding(input_dim=vocabulary_size, output_dim=hidden_dim, input_length=max_seq_len)(input)
    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input, outputs=x)
    return model

# hyper-parameters
hidden_dim = 32
# model construction
model = embedding_model(vocabulary_size+1, hidden_dim, max_seq_len)
print(model.summary())
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# fit the model
model.fit(sequences, np.array(labels), batch_size=4, epochs=5)

[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
embedding (Embedding)        (None, 4, 32)             480       
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 609
Trainable params: 609
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
3/3 [==============================] - 14s 5ms/step - loss: 0.6861 - acc: 0.6312
Epoch 2/5
3/3 